<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/fraud/response.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import json as js
import re

In [15]:
hospital = pd.read_csv('/content/drive/MyDrive/AIEngineer/fraud/health_office.csv')
hospital.columns

<ipython-input-15-d69fde0a6e6e>:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  hospital = pd.read_csv('/content/drive/MyDrive/AIEngineer/fraud/health_office.csv')


Index(['ชื่อ', 'รหัส 9 หลักใหม่', 'รหัส 9 หลัก', 'รหัส 5 หลัก',
       'เลขอนุญาตให้ประกอบสถานบริการสุขภาพ 11 หลัก', 'ประเภทองค์กร',
       'ประเภทหน่วยบริการสุขภาพ', 'สังกัด', 'แผนก/กรม', 'ระดับโรงพยาบาล',
       'เตียงที่ใช้จริง', 'สถานะการใช้งาน', 'เขตบริการ', 'ที่อยู่',
       'รหัสจังหวัด', 'จังหวัด', 'รหัสอำเภอ', 'อำเภอ/เขต', 'รหัสตำบล',
       'ตำบล/แขวง', 'หมู่', 'รหัสไปรษณีย์', 'แม่ข่าย', 'วันที่ก่อตั้ง',
       'วันที่ปิดบริการ', 'อัพเดตล่าสุด(เริ่ม 05/09/2566)'],
      dtype='object')

In [16]:
data = pd.read_csv('/content/drive/MyDrive/AIEngineer/fraud/opd_discharge_August_2024.csv')
cols_to_drop = [col for col in data.columns if col.endswith('_txt')]
data = data.drop(columns=cols_to_drop)
df = data[data['claim_status'].isin(['Approve', 'Decline'])]
print(df['claim_status'].value_counts())

claim_status
Approve    15581
Decline      184
Name: count, dtype: int64


In [17]:
df['hospital_code'] = df['hospital_code'].astype(str)
hospital['hospital_code'] = hospital['รหัส 5 หลัก'].str.replace('="', '').str.replace('"', '')
hospital = hospital[['hospital_code', 'ชื่อ']].rename(columns={'ชื่อ': 'hospital_name'})
df = df.merge(hospital, on='hospital_code', how='left')

<ipython-input-17-83a06c097e2e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hospital_code'] = df['hospital_code'].astype(str)


In [37]:
hospital[hospital['hospital_code'] == '40765']

,hospital_code,hospital_name
1394,40765,โรงพยาบาลทั่วไปขนาดใหญ่เกษมราษฎร์ รัตนาธิเบศร์


In [40]:
df[df['order_item'] != '[]'][['hospital_code', 'hospital_name', 'cid', 'insurance', 'claim_status', 'diagnosis', 'billing', 'order_item', 'response']]

In [76]:
decline = focus[focus['claim_status'].astype(str)=='Decline']
decline['insurance'].value_counts()

,count
insurance,
tli,23
aia,19
mtl,12
fwd,3


## **❗case: decline**
- **tli**: เข้าข้อยกเว้น, ICD10 อยู่ในกลุ่มโรค

- **aia**: ไม่สามารถเคลมได้ เนื่องจากสัญญาเพิ่มเติมสิ้นผลบังคับ, เบิกซ้ำซ้อนมากกว่า 1 ครั้ง/วัน, ต้องการข้อมูลทางการแพทย์เพิ่มเติม, ผลประโยชน์คุ้มครองเฉพาะค่ารักษาพยาบาลกรณีอุบัติเหตุ

- **mtl**: กรุณาสำรองจ่ายและส่งเอกสารการเคลมให้กับบริษัทฯโดยตรง, โรคมีระยะเวลาการรอคอย 180 วัน, ส่ง fax claim, เงื่อนไขโรคยกเว้นเฉพาะรายบุคคล, โรคที่เข้าข่ายในเงื่อนไขข้อยกเว้นกรมธรรม์สัญญาสุขภาพ

- **fwd**: copay == ผู้ป่วยต้องจ่ายเอง

In [ ]:
js.loads(decline[decline['insurance']=='fwd']['response'].iloc[2])

In [115]:
approve = focus[focus['claim_status'].astype(str)=='Approve']
approve['insurance'].value_counts()

,count
insurance,
mtl,2068
aia,1572
fwd,868
tli,265


In [120]:
js.loads(approve[approve['insurance']=='aia']['response'].iloc[0])

{'detail': [],
 'claim_no': 'C029322630',
 'insurer_detail': [{'claim_no': 'C029322630',
   'non_cover': '',
   'benefit_code': 'HS',
   'copay_amount': '',
   'coverage_code': 'ME สัญญาเพิ่มเติมมีสิทธิ์ใช้บริการเรียกร้องสินไหม',
   'excess_amount': '',
   'policy_number': 'P260294452',
   'insurance_type': '',
   'order_sequence': '1',
   'trailer_number': '',
   'approved_amount': '',
   'result_description': ''},
  {'claim_no': 'C029322630',
   'non_cover': '',
   'benefit_code': 'HS',
   'copay_amount': '',
   'coverage_code': 'ME ไม่สามารถใช้บริการเรียกร้องสินไหมได้',
   'excess_amount': '',
   'policy_number': 'P318183114',
   'insurance_type': '',
   'order_sequence': '2',
   'trailer_number': '',
   'approved_amount': '',
   'result_description': ''},
  {'claim_no': 'C029322630',
   'non_cover': '',
   'benefit_code': 'HS',
   'copay_amount': '',
   'coverage_code': 'ME สัญญาเพิ่มเติมมีสิทธิ์ใช้บริการเรียกร้องสินไหม',
   'excess_amount': '',
   'policy_number': 'P328402393',
  